In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import itertools

In [3]:
def uint8_to_binarray(integer):
  """Convert integer into fixed-length 8-bit binary array. LSB in [0].
  Extended and modified code from https://github.com/projf/display_controller/blob/master/model/tmds.py
  """

  b_array = [int(i) for i in reversed(bin(integer)[2:])]
  b_array += [0]*(8-len(b_array))
  return b_array


def binarray_to_uint(binarray):
	
  array = binarray[::-1]
  num = array[0]
  for n in range(1,len(binarray)):
    num = (num << 1) + array[n]

  return num

def TMDS_pixel (D,cnt=0):
  """8bit pixel TMDS coding

  Inputs: 
  - pix: 8-bit pixel
  - cnt: 0's and 1's balance. Default in 0 (balanced)

  Outputs:
  - pix_out: TDMS coded 16-bit pixel (only 10 useful)
  - cnt: 0's and 1's balance updated with new pixel coding

  """ 
  # Convert 8-bit pixel to binary list D
  #D = uint8_to_binarray(pix)

  # Initialize output q
  qm = [D[0]]

  # 1's unbalanced condition at current pixel
  N1_D = np.sum(D)

  if N1_D>4 or (N1_D==4 and not(D[0])):

    # XNOR of consecutive bits
    for k in range(1,8):
      qm.append( not(qm[k-1] ^ D[k]) )
    qm.append(0)

  else:
    # XOR of consecutive bits
    for k in range(1,8):
      qm.append( qm[k-1] ^ D[k] )
    qm.append(1)

  # Initialize output qout
  qout = qm.copy()

  # Unbalanced condition with previous and current pixels
  N1_qm = np.sum(qm[:8])
  N0_qm = 8 - N1_qm

  if cnt==0 or N1_qm==4:

    qout.append(not(qm[8]))
    qout[8] = qm[8]
    qout[:8]=qm[:8] if qm[8] else np.logical_not(qm[:8])

    if not(qm[8]):
      cnt += N0_qm - N1_qm 
    else:
      cnt += N1_qm - N0_qm 

  else:

    if (cnt>0 and N1_qm>4) or (cnt<0 and N1_qm<4):
      qout.append(1)
      qout[8] = qm[8]
      qout[:8] = np.logical_not(qm[:8])
      cnt += 2*qm[8] + N0_qm - N1_qm
    else:
      qout.append(0)
      qout[8] = qm[8]
      qout[:8] = qm[:8]
      cnt += -2*(not(qm[8])) + N1_qm - N0_qm

  return qout, cnt


In [7]:
enteros = range(-9, 10)
binarios = [list(map(int, list(bin(i)[2:].zfill(8)))) for i in range(256)]

combinaciones = list(itertools.product(binarios, enteros))

X = np.array([list(comb[0]) + [comb[1]] for comb in combinaciones])
X_float = X.astype(float)
Y = np.zeros((X.shape[0],10))
for i in range(X.shape[0]):
    Y[i,:],cnt_posta = TMDS_pixel(X[i,:8][::-1],X[i,8])
    print(f"X: {X[i]} Y: {Y[i]}")

X: [ 0  0  0  0  0  0  0  0 -9] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -8] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -7] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -6] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -5] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -4] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -3] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -2] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [ 0  0  0  0  0  0  0  0 -1] Y: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X: [0 0 0 0 0 0 0 0 0] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 1] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 2] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 3] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 4] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 5] Y: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
X: [0 0 0 0 0 0 0 0 6] Y: [0. 0. 0. 0. 0. 0. 0. 0

In [8]:
# Convertir los datos en tensores de PyTorch
X_t = torch.tensor(X, dtype=torch.float32)
Y_t = torch.tensor(Y, dtype=torch.float32)

In [12]:
# Definir una red neuronal simple
class Model_TMDS(nn.Module):
    def __init__(self):
        super(Model_TMDS, self).__init__()
        self.fc1 = nn.Linear(9, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [13]:
# Crear una instancia del modelo
TMDS = Model_TMDS()
# Definir función de pérdida y optimizador
criterion = nn.BCELoss()
optimizer = optim.Adam(TMDS.parameters(), lr=0.001)

In [22]:
# Entrenar el modelo (overfitting intencional)
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = TMDS(X)
    # Calcular la pérdida
    loss = criterion(outputs, Y)

    # Backward y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [100/10000], Loss: 0.5919
Epoch [200/10000], Loss: 0.4905
Epoch [300/10000], Loss: 0.4082
Epoch [400/10000], Loss: 0.3237
Epoch [500/10000], Loss: 0.2610
Epoch [600/10000], Loss: 0.2093
Epoch [700/10000], Loss: 0.1876
Epoch [800/10000], Loss: 0.1686
Epoch [900/10000], Loss: 0.1342
Epoch [1000/10000], Loss: 0.1116
Epoch [1100/10000], Loss: 0.1001
Epoch [1200/10000], Loss: 0.0947
Epoch [1300/10000], Loss: 0.0920
Epoch [1400/10000], Loss: 0.0904
Epoch [1500/10000], Loss: 0.0889
Epoch [1600/10000], Loss: 0.0819
Epoch [1700/10000], Loss: 0.0591
Epoch [1800/10000], Loss: 0.0351
Epoch [1900/10000], Loss: 0.0207
Epoch [2000/10000], Loss: 0.0130
Epoch [2100/10000], Loss: 0.0086
Epoch [2200/10000], Loss: 0.0060
Epoch [2300/10000], Loss: 0.0044
Epoch [2400/10000], Loss: 0.0034
Epoch [2500/10000], Loss: 0.0027
Epoch [2600/10000], Loss: 0.0021
Epoch [2700/10000], Loss: 0.0017
Epoch [2800/10000], Loss: 0.0015
Epoch [2900/10000], Loss: 0.0012
Epoch [3000/10000], Loss: 0.0011
Epoch [3100/10000],

In [14]:
# Guardar los pesos del modelo en un archivo .pth
torch.save(TMDS.state_dict(), 'TMDS.pth')
print("Los pesos del modelo se han guardado")

Los pesos del modelo se han guardado
